# Análisis de farmacias

Este notebook analiza datos de medicinas por persona, ingresos y pobreza en distintos municipios. Se incluye el ajuste de una distribución log-normal y se exploran correlaciones entre diferentes variables. También se busca encontrar los municipios más similares según ciertas características.

---

## 1. Importación de bibliotecas
Se cargan las bibliotecas necesarias para el análisis de datos y visualización.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from sklearn.metrics.pairwise import cosine_similarity
from scipy import stats
from scipy.stats import gaussian_kde
from tqdm import tqdm

ModuleNotFoundError: No module named 'seaborn'

## Cargar datos principales
Aquí cargamos los datos principales de las farmacias desde un archivo CSV.

In [3]:
df = pd.read_csv('../enigh_eda/joined_df.csv')

In [ ]:
df.head()

In [ ]:
df.columns

## Filtrado de datos de medicinas
Se filtran los datos para obtener solo aquellos donde haya consumo de medicinas por persona.

In [6]:
df_medicinas = df[df['medicina_por_persona']>0].reset_index(drop=True)

In [ ]:
df_medicinas.columns

In [ ]:
df_medicinas['medicina_por_persona']

## Análisis de correlaciones
Se calculan las correlaciones entre el ingreso por persona y el gasto en medicinas por persona.

In [ ]:
df_medicinas[['ingreso_por_persona_mensual','medicina_por_persona_mensual']].corr()

## Visualización de datos
Se grafican los datos de ingreso por persona y medicinas, así como su distribución.

In [ ]:
sns.scatterplot(x='medicina_por_persona_mensual',y='ingreso_por_persona_mensual',data=df_medicinas)
plt.show()

In [ ]:
sns.boxplot(x='medicina_por_persona_mensual',data=df_medicinas)
plt.show()

In [ ]:
df_medicinas['medicina_por_persona_mensual'].describe()

In [ ]:
sns.histplot(df_medicinas['medicina_por_persona_mensual'],bins=100)
plt.show()

## Transformación logarítmica
Se transforma la variable de medicinas por persona a una escala logarítmica y se visualiza su distribución.

In [14]:
df_medicinas['log_medicina_por_persona_mensual'] = np.log(df_medicinas['medicina_por_persona_mensual'])

In [ ]:
sns.histplot(df_medicinas['log_medicina_por_persona_mensual'],bins=100)
plt.show()

## Análisis de valores atípicos
Se filtran y visualizan los valores atípicos en el consumo de medicinas.

In [16]:
df_atypical = df_medicinas[df_medicinas['medicina_por_persona_mensual']>71+329].reset_index(drop=True)

In [ ]:
sns.histplot(df_atypical['medicina_por_persona_mensual'],bins=100)

In [ ]:
sns.boxplot(x='medicina_por_persona_mensual',data=df_atypical)

## Descripción del ingreso por persona
Se revisa la distribución del ingreso por persona.

In [ ]:
df_medicinas['ingreso_por_persona_mensual'].describe()

## Filtrado de datos por nivel de ingreso
Se filtran los datos para incluir solo aquellos municipios con ingresos por debajo de 3000.

In [20]:
df_atypical = df_medicinas[(df_medicinas['ingreso_por_persona_mensual']<3000) & (df_medicinas['ingreso_por_persona_mensual']>0)].reset_index(drop=True)

In [ ]:
df_atypical[['ingreso_por_persona_mensual','medicina_por_persona_mensual']].corr()

## Variables geográficas
Se seleccionan y visualizan las columnas geográficas relevantes.

In [ ]:
df_medicinas[['ubica_geo','NOM_ENT','NOM_MUN']]

In [ ]:
sns.histplot(df_medicinas[df_medicinas['ubica_geo']==8037], x='medicina_por_persona_mensual',bins=100)
plt.show()

In [ ]:
df_medicinas['ubica_geo'].nunique()

## Carga y transformación de datos de pobreza
Se carga el dataset de pobreza y se formatean las columnas relevantes.

In [25]:
df_pobreza = pd.read_csv('pobreza.csv', skiprows=2)

In [26]:
df_pobreza = df_pobreza.drop([0,1], axis=0)

In [27]:
df_medicinas['ubica_geo'] = df_medicinas['ubica_geo'].astype(str)

In [ ]:
df_medicinas['ubica_geo']

In [29]:
df_pobreza['Clave de municipio'] = df_pobreza['Clave de municipio'].astype(str)

In [30]:
df_pobreza['Clave de municipio'] = df_pobreza['Clave de municipio'].str.replace('.0','')

In [ ]:
df_pobreza.columns

## Unificación de datasets
Se unen los datasets de medicinas y pobreza.

In [32]:
pobreza_check = pd.merge(df_medicinas[['ubica_geo','medicina_por_persona_mensual','alimentos','ingreso_por_persona_mensual']], df_pobreza, left_on='ubica_geo', right_on='Clave de municipio', how='left')

In [ ]:
df_pobreza.columns

In [ ]:
pobreza_check

## Conversión de columnas relevantes
Se convierten las columnas de población y porcentaje a formato numérico.

In [35]:
pobreza_check['Población 2020*\n(leer nota al final del cuadro)'] = pobreza_check['Población 2020*\n(leer nota al final del cuadro)'].str.replace(".","").astype(int)

In [36]:
pobreza_check['Población 2020*\n(leer nota al final del cuadro)'] = pobreza_check['Población 2020*\n(leer nota al final del cuadro)'].astype(float)

In [37]:
pobreza_check['Porcentaje\n2020'] = pobreza_check['Porcentaje\n2020'].str.replace(",","").astype(float)

## Análisis de correlaciones estadísticas
Se calculan las correlaciones de Pearson entre las variables de interés, incluyendo el porcentaje de pobreza y consumo de medicinas.

In [ ]:
pobreza_check[['medicina_por_persona_mensual','alimentos','ingreso_por_persona_mensual','Población 2020*\n(leer nota al final del cuadro)','Porcentaje\n2020']].corr()

In [ ]:
x = pobreza_check['Porcentaje\n2020']
y = pobreza_check['medicina_por_persona_mensual']

corr, p_value = stats.pearsonr(x,y)

print(f'Pearson correlation coefficient: {corr}')
print(f'P-value: {p_value}')

In [ ]:
x = df_medicinas['ingreso_por_persona_mensual']
y = df_medicinas['medicina_por_persona_mensual']

corr, p_value = stats.pearsonr(x,y)

print(f'Pearson correlation coefficient: {corr}')
print(f'P-value: {p_value}')

Calcular el gasto en alimentos por persona al mes

In [42]:
df_medicinas['alimentos_por_persona_mensual'] = df_medicinas['alimentos']/df_medicinas['tot_integ']/3

Seleccionar columnas relevantes

In [43]:
relevant_df = df_medicinas[['ubica_geo','medicina_por_persona_mensual','alimentos_por_persona_mensual','ingreso_por_persona_mensual']]

Agrupar los datos por ubicación geográfica

In [44]:
grouped_mun = relevant_df.groupby('ubica_geo')

 Mostrar el promedio de gasto en medicina por persona al mes por municipio

In [ ]:
for a,b in grouped_mun:
    print(a)
    print(b['medicina_por_persona_mensual'].mean())

Crear DataFrame para almacenar los parámetros de las distribuciones log-normales

In [123]:
medicinas_pdf_df = pd.DataFrame(columns=['ubica_geo','shape','loc','scale'])

In [ ]:
df_medicinas['medicina_por_persona_mensual'].isna().sum()

In [ ]:
relevant_df.columns

Ajuste de distribuciones log-normales por municipio

In [ ]:
for ubica_geo, data in tqdm(grouped_mun, desc='Ajustando log-normal', unit='municipios'):
    # Si hay menos de 2 registros, se crea un conjunto de datos más grande usando multiplicadores aleatorios
    if len(data['medicina_por_persona_mensual']) < 2:
        # Crear 30 multiplicadores aleatorios entre 0.9 y 1.1
        random_multipliers = np.random.uniform(0.9, 1.1, size=30)

        # Repetir los datos originales para tener al menos 30 filas
        original_data_repeated = pd.concat([data] * (30 // len(data) + 1), ignore_index=True)
        
        # Ajustar el conjunto repetido para que tenga exactamente 30 filas
        original_data_repeated = original_data_repeated.iloc[:30]

        # Multiplicar la columna 'medicina_por_persona_mensual' por los multiplicadores aleatorios
        original_data_repeated['medicina_por_persona_mensual'] *= random_multipliers
        
        data = original_data_repeated 

    # Ajustar una distribución log-normal a los datos
    shape, loc, scale = stats.lognorm.fit(data['medicina_por_persona_mensual'])

    # Guardar los parámetros ajustados
    params = {'ubica_geo': ubica_geo, 'shape': shape, 'loc': loc, 'scale': scale}

    # Añadir los parámetros al DataFrame
    medicinas_pdf_df = pd.concat([medicinas_pdf_df, pd.DataFrame([params])], ignore_index=True)

# Guardar los resultados en un archivo CSV
medicinas_pdf_df.to_csv('medicinas_pdf.csv', index=False)

In [ ]:
medicinas_pdf_df.head()

Obtener los parámetros de la fila para una distribución log-normal

In [ ]:
shape = row['shape'].values[0]  # 'shape' o sigma
loc = row['loc'].values[0]      # parámetro loc
scale = row['scale'].values[0]  # parámetro scale

In [ ]:
# Generar valores de una distribución log-normal
x = np.linspace(0, 10, 1000)  # Crear un rango de valores x
y = stats.lognorm.pdf(x, s=shape, loc=loc, scale=scale)  # Función de densidad de probabilidad para la distribución log-normal

# Graficar la distribución
plt.plot(x, y, label=f'Distribución Log-Normal (id=1)')
plt.title('Distribución Log-Normal para id=1')
plt.xlabel('X')
plt.ylabel('Densidad de Probabilidad')
plt.legend()
plt.show()

In [ ]:
df_medicinas.columns

Seleccionar algunas columnas para análisis geográfico

In [ ]:
df_medicinas[['latitude','longitude','LAT_DECIMAL','LON_DECIMAL']]

Crear un nuevo DataFrame con datos de ubicación y población

In [133]:
df_mun = df_medicinas[['ubica_geo','CVE_ENT', 'AMBITO','LAT_DECIMAL', 'LON_DECIMAL',
       'POB_TOTAL', 'POB_MASCULINA', 'POB_FEMENINA',
       'TOTAL DE VIVIENDAS HABITADAS']]

In [ ]:
df_mun.dtypes

Cargar un archivo CSV con datos de todos los municipios

In [135]:
df_todos_municipios = pd.read_csv('../enigh_eda/AGEEML_20249151733738.csv', encoding='latin1')

Filtrar para quedarse solo con localidades

In [136]:
df_todos_municipios = df_todos_municipios[df_todos_municipios['CVE_LOC'] == 1].reset_index(drop=True)

Ajustar las claves de municipio y entidad para tener longitud estándar

In [137]:
df_todos_municipios['CVE_MUN'] = df_todos_municipios['CVE_MUN'].astype(str).str.zfill(3)
df_todos_municipios['CVE_ENT'] = df_todos_municipios['CVE_ENT'].astype(str).str.zfill(2)

Crear columna de ubicación geográfica combinando las claves de entidad y municipio

In [138]:
df_todos_municipios['ubica_geo'] = df_todos_municipios['CVE_ENT'] + df_todos_municipios['CVE_MUN']

In [139]:
df_mun_relevant = df_todos_municipios[['ubica_geo','CVE_ENT', 'AMBITO','LAT_DECIMAL', 'LON_DECIMAL',
       'POB_TOTAL', 'POB_MASCULINA', 'POB_FEMENINA',
       'TOTAL DE VIVIENDAS HABITADAS']]

Eliminar duplicados basados en la columna de ubicación geográfica

In [140]:
df_mun_relevant = df_mun_relevant.drop_duplicates(subset='ubica_geo')

In [ ]:
df_mun_relevant.head()

Eliminar ceros a la izquierda en la columna de ubicación geográfica

In [142]:
df_mun = df_mun.drop_duplicates(subset='ubica_geo')

In [ ]:
df_mun

In [144]:
df_mun_relevant['ubica_geo'] = df_mun_relevant['ubica_geo'].str.lstrip('0')

In [145]:
filtered_df_mun_estimate = df_mun_relevant[~df_mun_relevant['ubica_geo'].isin(df_mun['ubica_geo'])]

In [146]:
df_encoded_infer = pd.get_dummies(filtered_df_mun_estimate,columns=['AMBITO','CVE_ENT'])

In [147]:
df_encoded_infer = df_encoded_infer.reset_index(drop=True)

In [ ]:
df_encoded_infer.dtypes

In [ ]:
df_encoded_infer = df_encoded_infer.applymap(lambda x: pd.to_numeric(x, errors='coerce'))

In [150]:
df_encoded_infer = df_encoded_infer.astype('float')

In [152]:
# Función para encontrar IDs más similares usando similitud coseno
def find_most_similar_ids(df_a, df_b):
    # Extraer los IDs de ambos DataFrames
    ids_a = df_a['ubica_geo']
    ids_b = df_b['ubica_geo']
    
    # Eliminar las columnas de IDs para calcular la similitud
    features_a = df_a.drop(columns=['ubica_geo']).fillna(0)
    features_b = df_b.drop(columns=['ubica_geo']).fillna(0)
    
    # Calcular la similitud coseno entre cada fila de A y B
    similarity_matrix = cosine_similarity(features_a, features_b)
    
    # Encontrar el índice de la fila más similar en B para cada fila en A
    most_similar_indices = similarity_matrix.argmax(axis=1)
    
    # Mapear estos índices a los IDs correspondientes en B
    most_similar_ids = ids_b.iloc[most_similar_indices].values
    
    # Crear un DataFrame de resultados que muestre el ID más similar para cada fila de A
    result_df = pd.DataFrame({
        'id_A': ids_a,
        'most_similar_id_B': most_similar_ids
    })
    
    return result_df

Convertimos las columnas categóricas 'AMBITO' y 'CVE_ENT' de 'df_mun' en variables dummy, y convertimos el DataFrame resultante a tipo 'float'.

In [153]:
df_mun_query = pd.get_dummies(df_mun,columns=['AMBITO','CVE_ENT']).astype('float')

In [ ]:
df_mun_query.shape

In [ ]:
df_mun_query.columns

In [ ]:
df_encoded_infer.shape

In [ ]:
df_encoded_infer.columns

Añadimos nuevas columnas 'CVE_ENT_03', 'CVE_ENT_06' y 'CVE_ENT_09' a 'df_encoded_infer' y las inicializamos con el valor 0.

In [158]:
df_encoded_infer['CVE_ENT_03'] = 0
df_encoded_infer['CVE_ENT_06'] = 0
df_encoded_infer['CVE_ENT_09'] = 0

In [ ]:
df_encoded_infer.shape

Verificamos cuántos registros en la columna 'ubica_geo' de 'df_mun_query' existen también en la columna 'ubica_geo' de 'df_medicinas' (convertidos a tipo float).

In [ ]:
df_mun_query['ubica_geo'].isin(df_medicinas['ubica_geo'].astype(float)).sum()

Utilizamos la función 'find_most_similar_ids' para encontrar los IDs más similares entre 'df_encoded_infer' y 'df_mun_query'.

In [161]:
resulting_df = find_most_similar_ids(df_encoded_infer, df_mun_query)

In [ ]:
resulting_df

Convertimos las columnas 'id_A' y 'most_similar_id_B' a tipo string y eliminamos los decimales '.0' que fueron añadidos al convertir.

In [163]:
resulting_df['id_A'] = resulting_df['id_A'].astype(str).str.replace('.0','')
resulting_df['most_similar_id_B'] = resulting_df['most_similar_id_B'].astype(str).str.replace('.0','')

In [ ]:
resulting_df

Comprobamos cuántos IDs en la columna 'most_similar_id_B' existen también en la columna 'ubica_geo' del DataFrame 'df_mun' (convertidos a tipo string).

In [ ]:
resulting_df['most_similar_id_B'].isin(df_mun['ubica_geo'].astype(str)).sum()

Realizamos una fusión (merge) entre 'resulting_df' y 'medicinas_pdf_df', uniendo con la columna 'most_similar_id_B' de 'resulting_df' y 'ubica_geo' de 'medicinas_pdf_df'.
Posteriormente eliminamos las columnas 'ubica_geo' y 'most_similar_id_B', y renombramos la columna 'id_A' como 'ubica_geo'.

In [166]:
new_mun = pd.merge(resulting_df,medicinas_pdf_df, left_on='most_similar_id_B', right_on='ubica_geo').drop(columns = ['ubica_geo','most_similar_id_B']).rename(columns={'id_A':'ubica_geo'})

In [ ]:
new_mun.dtypes

Comprobamos cuántos valores en la columna 'ubica_geo' de 'new_mun' existen también en la columna 'ubica_geo' de 'medicinas_pdf_df' (convertidos a tipo string).

In [ ]:
new_mun['ubica_geo'].isin(medicinas_pdf_df['ubica_geo'].astype(str)).sum()

Concatenamos el DataFrame 'new_mun' con 'medicinas_pdf_df' y guardamos el resultado

In [169]:
medicinas_pdf_df_new = pd.concat([medicinas_pdf_df,new_mun], ignore_index=True)

In [170]:
medicinas_pdf_df_new.to_csv('medicinas_pdf.csv', index=False)